In [23]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-70b-8192",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    groq_api_key = "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"
    # other params...
)

In [24]:
response = llm.invoke("The first person to land on moon . . . ")
print(response.content)

That's an easy one!

The first person to set foot on the Moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the Moon's surface on July 20, 1969, during the Apollo 11 mission.

Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on another celestial body.

He was followed by fellow astronaut Edwin "Buzz" Aldrin, who also walked on the Moon during the mission.


In [36]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-44232?from=job%20search%20funnel")
page_data  =  loader.load().pop().page_content

print(page_data)

Apply for Senior Manager, Software Engineering Innovation

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySenior Manager, Software Engineering InnovationBeaverton, OregonBecom

In [38]:
from langchain_core.prompts import PromptTemplate

# Instantiation using from_template (recommended)
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTIONS:
    The scraped text is from the career's page of a website. 
    Your job is to extract the job postings and return them in JSON format containing the following keys: 'role', 'experience', 'skills', 'description'. 
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

# Create the LLM chain
chain_extract = prompt_extract | llm

# Provide the input data to the chain and invoke it
res = chain_extract.invoke(input={'page_data': page_data})

# Output the result
print(res.content)

Here is the extracted job posting in JSON format:

```
{
  "role": "Senior Manager, Software Engineering Innovation",
  "experience": "10+ years software engineering experience, 5+ years leading full stack DevOps teams, 3+ years in a leadership role with direct reports",
  "skills": [
    "Domain experience building data-centric software products in sports research, medical data, or related field",
    "Excellent verbal and written communication skills, both technical and business-facing",
    "Solid computer science fundamentals, including algorithms and data structures",
    "Fluent in modern software architectural principles and patterns (REST, domain-driven design, microservices, etc)",
    "Experience with distributed cloud systems tech (AWS, node.js, EC2, Lambda, DynamoDB, Elasticsearch, or Google Cloud/Azure equivalents)",
    "Experience with front-end web application tech (javascript, CSS, html5, Vue, React)",
    "Experience delivering data products to users in a lakehouse pl

In [40]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Senior Manager, Software Engineering Innovation',
 'experience': '10+ years software engineering experience, 5+ years leading full stack DevOps teams, 3+ years in a leadership role with direct reports',
 'skills': ['Domain experience building data-centric software products in sports research, medical data, or related field',
  'Excellent verbal and written communication skills, both technical and business-facing',
  'Solid computer science fundamentals, including algorithms and data structures',
  'Fluent in modern software architectural principles and patterns (REST, domain-driven design, microservices, etc)',
  'Experience with distributed cloud systems tech (AWS, node.js, EC2, Lambda, DynamoDB, Elasticsearch, or Google Cloud/Azure equivalents)',
  'Experience with front-end web application tech (javascript, CSS, html5, Vue, React)',
  'Experience delivering data products to users in a lakehouse platform (databricks strongly preferred)',
  'Experience delivering Gen AI appl

In [41]:
type(json_res)

dict

In [42]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [49]:
import chromadb
import uuid

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name = "portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
        metadatas = {"links" : row["Links"]},
        ids = [str(uuid.uuid4())])


In [51]:
links = collection.query(query_texts = ["Experience in Python", "Properties in React"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [52]:
job = json_res
job['skills']

['Domain experience building data-centric software products in sports research, medical data, or related field',
 'Excellent verbal and written communication skills, both technical and business-facing',
 'Solid computer science fundamentals, including algorithms and data structures',
 'Fluent in modern software architectural principles and patterns (REST, domain-driven design, microservices, etc)',
 'Experience with distributed cloud systems tech (AWS, node.js, EC2, Lambda, DynamoDB, Elasticsearch, or Google Cloud/Azure equivalents)',
 'Experience with front-end web application tech (javascript, CSS, html5, Vue, React)',
 'Experience delivering data products to users in a lakehouse platform (databricks strongly preferred)',
 'Experience delivering Gen AI applications (to production preferred, at minimum POC to business users required)']

In [53]:

prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineering Innovation Leadership for Your Organization

Dear Hiring Manager,

I came across your job posting for a Senior Manager, Software Engineering Innovation, and I'm excited to introduce AtliQ, a seasoned AI & Software Consulting company, as a potential partner to fulfill your needs. Our team of experts has a proven track record of delivering innovative software solutions that drive business growth and efficiency.

With our extensive experience in software engineering, DevOps, and leadership, I believe we can provide the ideal candidate to lead your development of data and sports research technology capabilities. Our team members possess the required skills, including:

* Domain experience in building data-centric software products
* Excellent communication skills, both technical and business-facing
* Solid computer science fundamentals and fluency in modern software architectural principles
* Experience with distributed cloud systems, front-end web appl